In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, InputLayer
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam



import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [ ]:
### Analyse and selective augment data
###Read input format section in Protein−Ligand Scoring with Convolutional Neural Networks
 

In [7]:
###Pickle data

In [38]:
### Genetate fake data
fake_X_data = np.random.rand(60,48,48,48)
fake_Y_data = np.random.rand(60)

In [39]:
###Normalise image data 

In [40]:
###Shuffle
permutation = np.random.permutation(fake_X_data.shape[0])
X = fake_X_data[permutation]
Y = fake_Y_data[permutation]

In [41]:
####Train test split
train_upper = int(np.around((fake_X_data.shape[0]*0.9),0))

X_train = fake_X_data[:train_upper]
X_test = fake_X_data[train_upper:]

Y_train = fake_Y_data[:train_upper]
Y_test = fake_Y_data[train_upper:]

In [42]:
X_train.shape

(54, 48, 48, 48)

In [47]:
def build_CNN():
    model = Sequential(name='CNN')
    
    # elu=Exponential Linear Unit, similar to leaky Relu
    #perhaps normalise in layer 1 
    
    # Convolution Layers
    model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2],X_train.shape[3])))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(32, (2, 2), strides=(1, 1), activation='elu'))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(64, (2, 2), strides=(1, 1), activation='elu'))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(128, (2, 2), strides=(1, 1), activation='elu'))
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.1))
    #change to sigmoid if pic50 between 0 and 1
    model.add(Dense(2,activation = 'linear'))
    
     
    
  
    optimizer = Adam(learning_rate=1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    
    return model

model = build_CNN()
print(model.summary())

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_14 (MaxPooling (None, 24, 24, 48)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 23, 23, 32)        6176      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 10, 10, 64)        8256      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 4, 4, 128)         32896     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0       